In [1]:
from datetime import datetime
import polars as pl
import os
import sys
import plotly.express as px
import polars.selectors as cs


PROJECT_PATH = os.path.abspath(os.path.join("..", ".."))
PIPELINE_PATH = os.path.join(PROJECT_PATH, "pipeline")
DATA_DIRECTORY = os.path.join(PROJECT_PATH, "data")

if PIPELINE_PATH not in sys.path:
    sys.path.append(PIPELINE_PATH)
    
from utils.paths import PROCESSED_PICARRO_DATA_DIRECTORY
from utils.calibration_processing import process_bottle

assert(os.path.exists(DATA_DIRECTORY))
assert(os.path.exists(PROCESSED_PICARRO_DATA_DIRECTORY))

In [2]:
start_date = datetime(2023, 12, 19, 0, 0, 0)
end_date = datetime(2023, 12 , 23, 0, 0, 0)
  
# Load Picarro Data
# DWD Picarro: "Calibrated_1_min_DWD_Picarro_G2301_413.parquet"
df_p = pl.scan_parquet(os.path.join(PROCESSED_PICARRO_DATA_DIRECTORY,"Calibrated_1_min_DWD_Picarro_G2301_413.parquet"))
df_p = df_p.filter(pl.col("datetime").is_between(start_date, end_date)) \
    .select(["datetime", "picarro_corrected", "h2o_reported"]) \
    .collect()
    
df_p.head(1).vstack(df_p.tail(1))

datetime,picarro_corrected,h2o_reported
datetime[ms],f64,f64
2023-12-19 00:00:00,519.542796,0.874734
2023-12-23 00:00:00,428.267135,1.181619


In [3]:
def plot_sensor_measurement(df, col_name: str, filter = None):
    if filter != None:
        df = df.groupby_dynamic("datetime", every=filter).agg(cs.numeric().mean())
        
    fig = px.line(
        df,
        x="datetime",
        y=col_name,
        markers=True,
        title=col_name,
    )
    fig.show()
    
def process_measurement(start_date, end_date, plot=False):
    df_p_bottle = df_p.filter(pl.col("datetime").is_between(start_date, end_date))

    data = df_p_bottle.select(pl.col("picarro_corrected")).to_series().to_list()
    data_cut = data[int(len(data)*0.3):int(len(data)*0.95)]

    median = process_bottle(data_cut, ignore_len=True) 

    print(f"Picarro Median Reading: {median}")

    # Plots
    if plot:
        plot_sensor_measurement(df_p_bottle, col_name="picarro_corrected")
        plot_sensor_measurement(df_p_bottle, col_name="h2o_reported")

# Bottles on 19.12.2023

In [45]:
# bottle 41

# PICARRO
# 41		16:02	16:27	399,8
start_date = datetime(2023, 12, 19, 14, 56, 0)
end_date = datetime(2023, 12, 19, 15, 20, 0)

process_measurement(start_date, end_date, plot = True)

Picarro Median Reading: 402.33845708226943


In [5]:
# 42		16:32	16:53	401.7
start_date = datetime(2023, 12, 19, 15, 28, 0)
end_date = datetime(2023, 12, 19, 15, 47, 0)

process_measurement(start_date, end_date)

Picarro Median Reading: 404.28084224654657


In [6]:
# 43		16:55	17:15	401,5
start_date = datetime(2023, 12, 19, 15, 51, 0)
end_date = datetime(2023, 12, 19, 16, 9, 0)

process_measurement(start_date, end_date)

Picarro Median Reading: 404.09465945204835


In [7]:
# 44		17:18	17:40	401,7
start_date = datetime(2023, 12, 19, 16, 14, 0)
end_date = datetime(2023, 12, 19, 16, 34, 0)

process_measurement(start_date, end_date)

Picarro Median Reading: 404.1662843361729


In [8]:
# 45		17:41	18:02	403,6
start_date = datetime(2023, 12, 19, 16, 36, 0)
end_date = datetime(2023, 12, 19, 16, 56, 0)

process_measurement(start_date, end_date)

Picarro Median Reading: 406.27310849017164


In [9]:
# 46		18:04	18:24	403,2
start_date = datetime(2023, 12, 19, 16, 59, 0)
end_date = datetime(2023, 12, 19, 17, 18, 0)

process_measurement(start_date, end_date)

Picarro Median Reading: 405.8415053858322


In [10]:
# 47	20.12.2023	10:49	11:09	400,2
start_date = datetime(2023, 12, 20, 9, 43, 0)
end_date = datetime(2023, 12, 20, 10, 3, 0)

process_measurement(start_date, end_date)

Picarro Median Reading: 402.7636985592384


In [11]:
# 48		11:10	11:30	401,4
start_date = datetime(2023, 12, 20, 10, 5, 0)
end_date = datetime(2023, 12, 20, 10, 24, 0)

process_measurement(start_date, end_date)

Picarro Median Reading: 403.8434525836858


In [12]:
# 49		11:31	11:51   401,3
start_date = datetime(2023, 12, 20, 10, 26, 0)
end_date = datetime(2023, 12, 20, 10, 45, 0)

process_measurement(start_date, end_date)

Picarro Median Reading: 404.00236047057155


Corrected Picarro Timeshift on Windows PC

In [13]:
# 50		13:54	14:14	401,4
start_date = datetime(2023, 12, 20, 13, 45, 0)
end_date = datetime(2023, 12, 20, 14, 14, 0)

process_measurement(start_date, end_date)

Picarro Median Reading: 403.98960507209415


In [14]:
# 51		14:15	14:35	399,9
start_date = datetime(2023, 12, 20, 14, 15, 0)
end_date = datetime(2023, 12, 20, 14, 34, 0)

process_measurement(start_date, end_date)


Picarro Median Reading: 402.4487098262509


In [15]:
# 52		14:35	14:50	401,7
start_date = datetime(2023, 12, 20, 14, 35, 0)
end_date = datetime(2023, 12, 20, 14, 50, 0)

process_measurement(start_date, end_date)

Picarro Median Reading: 404.3338211154601


In [16]:
# 53		14:57	15:17	401,4
start_date = datetime(2023, 12, 20, 14, 57, 0)
end_date = datetime(2023, 12, 20, 15, 17, 0)

process_measurement(start_date, end_date)

Picarro Median Reading: 404.07819705002055


In [17]:
# 54		15:19	15:39	401,1
start_date = datetime(2023, 12, 20, 15, 19, 0)
end_date = datetime(2023, 12, 20, 15, 39, 0)

process_measurement(start_date, end_date)

Picarro Median Reading: 403.74321644442495


In [18]:
# 55		15:40	16:00	403,4
start_date = datetime(2023, 12, 20, 15, 40, 0)
end_date = datetime(2023, 12, 20, 16, 0, 0)

process_measurement(start_date, end_date)

Picarro Median Reading: 405.99519469080195


In [19]:
# 56		16:01	16:20	401,4
start_date = datetime(2023, 12, 20, 16, 2, 0)
end_date = datetime(2023, 12, 20, 16, 19, 0)

process_measurement(start_date, end_date)

Picarro Median Reading: 403.9337850829329


In [20]:
# 57		10:24	10:44	403,3
start_date = datetime(2023, 12, 21, 10, 25, 0)
end_date = datetime(2023, 12, 21, 10, 44, 0)

process_measurement(start_date, end_date)

Picarro Median Reading: 405.8811432314349


In [21]:
# 58		10:45	11:05	403,1
start_date = datetime(2023, 12, 21, 10, 45, 0)
end_date = datetime(2023, 12, 21, 11, 5, 0)

process_measurement(start_date, end_date)

Picarro Median Reading: 405.71712692843624


In [22]:
# 59		11:06	11:26	401,4
start_date = datetime(2023, 12, 21, 11, 6, 0)
end_date = datetime(2023, 12, 21, 11, 25, 0)

process_measurement(start_date, end_date)

Picarro Median Reading: 403.97594446019167


In [23]:
# 60		11:27	11:47	403,1
start_date = datetime(2023, 12, 21, 11, 27, 0)
end_date = datetime(2023, 12, 21, 11, 47, 0)

process_measurement(start_date, end_date)

Picarro Median Reading: 405.81767395442


In [24]:
# 61		11:48	12:08	522,4
start_date = datetime(2023, 12, 21, 11, 48, 0)
end_date = datetime(2023, 12, 21, 12, 7, 0)

process_measurement(start_date, end_date)

Picarro Median Reading: 525.8090878887841


In [25]:
# 62		13:26	13:46	522,6
start_date = datetime(2023, 12, 21, 13, 26, 0)
end_date = datetime(2023, 12, 21, 13, 45, 0)

process_measurement(start_date, end_date)

Picarro Median Reading: 525.8837538620087


In [26]:
# 63		13:48	14:08	518,4
start_date = datetime(2023, 12, 21, 13, 48, 0)
end_date = datetime(2023, 12, 21, 14, 7, 0)

process_measurement(start_date, end_date)

Picarro Median Reading: 521.6663325435406


In [27]:
# 64		14:09	14:29	522,3
start_date = datetime(2023, 12, 21, 14, 9, 0)
end_date = datetime(2023, 12, 21, 14, 29, 0)

process_measurement(start_date, end_date)

Picarro Median Reading: 525.6308581103898


In [28]:
# 65		14:30	14:50	521,0	
start_date = datetime(2023, 12, 21, 14, 31, 0)
end_date = datetime(2023, 12, 21, 14, 50, 0)

process_measurement(start_date, end_date)

Picarro Median Reading: 524.2813444300078


In [29]:
# 66		14:52	15:12	520,7	
start_date = datetime(2023, 12, 21, 14, 51, 30)
end_date = datetime(2023, 12, 21, 15, 11, 0)

process_measurement(start_date, end_date)

Picarro Median Reading: 524.0564396007776


In [30]:
# 67		15:13	15:33	522,2	
start_date = datetime(2023, 12, 21, 15, 13, 0)
end_date = datetime(2023, 12, 21, 15, 32, 0)

process_measurement(start_date, end_date)

Picarro Median Reading: 525.6138988273221


In [31]:
# 68		15:34	15:54	522,5	
start_date = datetime(2023, 12, 21, 15, 34, 0)
end_date = datetime(2023, 12, 21, 15, 53, 0)

process_measurement(start_date, end_date)

Picarro Median Reading: 525.7725659489872


In [32]:
# 69		15:55	16:15	522,1	
start_date = datetime(2023, 12, 21, 15, 55, 0)
end_date = datetime(2023, 12, 21, 16, 14, 0)

process_measurement(start_date, end_date)

Picarro Median Reading: 525.4162006688188


In [33]:
# 70		16:16	16:36	521,7	
start_date = datetime(2023, 12, 21, 16, 17, 0)
end_date = datetime(2023, 12, 21, 16, 35, 0)

process_measurement(start_date, end_date)

Picarro Median Reading: 525.0034711775301


In [34]:
# 71		16:41	17:00	521,6	
start_date = datetime(2023, 12, 21, 16, 41, 0)
end_date = datetime(2023, 12, 21, 16, 59, 0)

process_measurement(start_date, end_date)

Picarro Median Reading: 524.9285100804026


In [35]:
# 72		10:34	10:54	521,0
start_date = datetime(2023, 12, 22, 9, 34, 0)
end_date = datetime(2023, 12, 22, 9, 53, 0)

process_measurement(start_date, end_date)

Picarro Median Reading: 524.3075122568364


In [36]:
# 73		10:55	11:16	520,9
start_date = datetime(2023, 12, 22, 9, 55, 0)
end_date = datetime(2023, 12, 22, 10, 16, 0)

process_measurement(start_date, end_date)

Picarro Median Reading: 524.3280140678185


In [37]:
# 74		11:17	11:39	520,3	
start_date = datetime(2023, 12, 22, 10, 17, 0)
end_date = datetime(2023, 12, 22, 10, 38, 0)

process_measurement(start_date, end_date)

Picarro Median Reading: 523.6397110932725


In [38]:
# 75		12:48	13:08	521,1	
start_date = datetime(2023, 12, 22, 11, 48, 0)
end_date = datetime(2023, 12, 22, 12, 8, 0)

process_measurement(start_date, end_date)

Picarro Median Reading: 524.4249224244528


In [39]:
# 76		13:16	13:36	521,6	
start_date = datetime(2023, 12, 22, 12, 16, 0)
end_date = datetime(2023, 12, 22, 12, 35, 0)

process_measurement(start_date, end_date)

Picarro Median Reading: 524.8720877640983


In [40]:
# 77		13:37	13:58	521,2	
start_date = datetime(2023, 12, 22, 12, 37, 0)
end_date = datetime(2023, 12, 22, 12, 57, 0)

process_measurement(start_date, end_date)

Picarro Median Reading: 524.4989332749016


In [41]:
# 78		14:00	14:21	518,4	
start_date = datetime(2023, 12, 22, 13, 0, 0)
end_date = datetime(2023, 12, 22, 13, 21, 0)

process_measurement(start_date, end_date)

Picarro Median Reading: 521.7572171744932


In [42]:
# 79		14:23	14:43	520,6	
start_date = datetime(2023, 12, 22, 13, 23, 0)
end_date = datetime(2023, 12, 22, 13, 43, 0)

process_measurement(start_date, end_date)

Picarro Median Reading: 523.8961785786234


In [43]:
# 80		14:44	15:04	520,6	
start_date = datetime(2023, 12, 22, 13,45, 0)
end_date = datetime(2023, 12, 22, 14, 4, 0)

process_measurement(start_date, end_date)

Picarro Median Reading: 523.0029524916635
